# ResNet Recognition Model

by Flávia Carvalhido

### Train
- input (2): path to the image folder, path to the csv with image paths and classes
- output (1): trained model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import copy
import pandas as pd 
import shutil

cudnn.benchmark = True

Code adapted from PyTorch Tutorials : https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [ ]:
class ResNetTrain:
    def __init__(self, img_folder_path="./train", img_csv="./train.csv"):
        self.img_folder_path = img_folder_path
        self.img_csv = img_csv

        # Create directory with needed structure img_dir/class/img
        os.mkdir('./temp_train')
        df = pd.read_csv(self.img_csv)

        self.class_names = df[df.columns[1]].unique()

        for n in self.class_names:
            os.mkdir('./temp_train/'+n)

        for _, row in df.iterrows():
            shutil.move(row[0], os.path.join('./temp_train/',row[1]))


        # Preprocess images (same for test)
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [
                                     0.229, 0.224, 0.225])
            ]),
        }

        image_datasets = {x: datasets.ImageFolder(os.path.join(self.img_folder_path, x),
                                                data_transforms[x])
                        for x in ['train']}
        self.dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                    shuffle=True, num_workers=4)
                    for x in ['train']}
        self.dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}
        self.class_names = image_datasets['train'].classes #override

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    def train(self): 

        model_ft = models.resnet18(pretrained=True)
        num_ftrs = model_ft.fc.in_features
        # size of each output sample is set to the number of classes
        model_ft.fc = nn.Linear(num_ftrs, len(self.class_names))

        model_ft = model_ft.to(self.device)

        criterion = nn.CrossEntropyLoss()

        # Observe that all parameters are being optimized
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

        best_model_wts = copy.deepcopy(model_ft.state_dict())
        best_acc = 0.0

        for epoch in range(25):
            print(f'Epoch {epoch}/{25 - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model_ft.train()  # Set model to training mode
                else:
                    model_ft.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in self.dataloaders[phase]:
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)

                    # zero the parameter gradients
                    optimizer_ft.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model_ft(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer_ft.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    exp_lr_scheduler.step()

                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_acc = running_corrects.double() / self.dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model_ft.state_dict())

         

        # load best model weights
        model_ft.load_state_dict(best_model_wts)
        return model_ft


    def help(self):
        print("Class ResNetTrain will train a ResNet recognition model with the provided arguments (in order): the training images folder path and the training images csv path. After initializing the class with the previously mentioned arguments, just call the method \"train\" and a trained ResNet model will be returned.")

### Test 

- input (3),  trained model, path to the image folder, path to the csv with image paths and [optional: classes]
- output (2): 2D matrix of predictions (floats) for each image and class,  [optional, if ground truth is provided: accuracy]